In [1]:
from kafka import KafkaConsumer

# Kafka 서버와 연결
consumer = KafkaConsumer(
    'heat_pf',  # 가져올 Topic 이름
    bootstrap_servers="spark-worker-01:9092,spark-worker-02:9092,spark-worker-03:9092",
    group_id='pf_group'
)

In [2]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

token = os.environ.get("INFLUXDB_TOKEN")

org = "pfss"
url = "http://spark-master-01:8086/"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [ ]:
import json
from datetime import datetime



# Write API 생성
write_api = client.write_api(write_options=SYNCHRONOUS)

# 데이터 쓰기
for msg in consumer:
    # Consumer로부터 가져온 데이터 JSON 파싱
    data= json.loads(msg.value.decode())
    
    # InfluxDB에 쓸 데이터 생성
    influx_data = [
        {
            "measurement": "sensor_measurement",
            "tags": {
                "sensor_id": data['sensor_id']
            },
            "time": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
            "fields": {
                "temperature": data['온도'],
                "pf": data['pf'],
                "R_pf": data['R_pf'],
                "S_pf": data['S_pf'],
                "T_pf": data['T_pf'],
                "pred": data['pred'],
            }
        }
    ]
    # InfluxDB에 데이터 쓰기
    write_api.write(bucket="heat_pf", record=influx_data)